## Imports

In [1]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate();
# using Piccolo
using PiccoloQuantumObjects
using QuantumCollocation
using ForwardDiff
using LinearAlgebra
using LaTeXStrings
using SparseArrays
using Statistics
using CairoMakie
using Random
using NamedTrajectories

  Activating project at `~/Documents/research/pulses/project/notebooks/src`
┌ Warning: Circular dependency detected.
│ Precompilation will be skipped for dependencies in this cycle:
│  ┌ Piccolissimo
│  └─ QuantumCollocation
└ @ Base.Precompilation precompilation.jl:651
┌ Warning: Circular dependency detected.
│ Precompilation will be skipped for dependencies in this cycle:
│  ┌ Piccolissimo
│  └─ QuantumCollocation
└ @ Base.Precompilation precompilation.jl:651
┌ Warning: Replacing docs for `QuantumCollocation.ProblemTemplates.UnitaryUniversalProblem :: Union{}` in module `QuantumCollocation.ProblemTemplates`
└ @ Base.Docs docs/Docs.jl:243
ERROR: Method overwriting is not permitted during Module precompilation. Use `__precompile__(false)` to opt-out of precompilation.
┌ Warning: Replacing docs for `QuantumCollocation.ProblemTemplates.UnitaryUniversalProblem :: Union{}` in module `QuantumCollocation.ProblemTemplates`
└ @ Base.Docs docs/Docs.jl:243


In [2]:
# Problem parameters
T = 40
Δt = 0.8
U_goal = GATES.H
H_drive = [PAULIS.X, PAULIS.Y, PAULIS.Z]
piccolo_opts = PiccoloOptions(verbose=false)
pretty_print(X::AbstractMatrix) = Base.show(stdout, "text/plain", X);
sys = QuantumSystem(H_drive)
seeds = rand(1:1000, 25)
F=0.9999
num_iter = 6000
hess = false
hess_iter = 120
Qs = 10 .^ range(-4.0, 1.0, length=25)
a_bound = 1.0
dda_bound = 0.5
R=5e-3

0.005

In [3]:
Qs[end]

10.0

In [4]:
using JLD2, FileIO
using CairoMakie
using Statistics

# Directories where data is saved
var_dir = "artifacts/var_gap_data_export"
tog_dir = "artifacts/tog_gap_data_export"

# Load the first seed file to get metadata
var_first = load(joinpath(var_dir, "var_probs_seed_idx_1.jld2"))
tog_first = load(joinpath(tog_dir, "htog_probs_seed_idx_1.jld2"))

# Extract common parameters
Qs = var_first["Qs"]

n_seeds = length(readdir(var_dir)) - 1  # Subtract 1 for plot.png

# Initialize arrays to store loaded data
var_probs = Matrix{Any}(undef, n_seeds, length(Qs))
htog_probs = Matrix{Any}(undef, n_seeds, length(Qs))

# Load all var_probs data
for i in 1:n_seeds
    data = load(joinpath(var_dir, "var_probs_seed_idx_$(i).jld2"))
    var_probs[i, :] = data["var_probs"]
end

# Load all htog_probs data
for i in 1:n_seeds
    data = load(joinpath(tog_dir, "htog_probs_seed_idx_$(i).jld2"))
    htog_probs[i, :] = data["htog_probs"]
end

println("Loaded data for $(n_seeds) seeds across $(length(Qs)) Q values")

Loaded data for 25 seeds across 25 Q values


In [5]:
n_seeds

25

In [6]:
Qs = Qs[1:end-4]
var_probs = var_probs[:, 1:end-4]
tog_probs = htog_probs[:, 1:end-4]

25×21 Matrix{Any}:
 DirectTrajOptProblem
   timesteps            = 40
   duration             = 31.2
   variable names       = (:Ũ⃗, :a, :da, :dda, :Δt)
   knot point dimension = 18
  …  DirectTrajOptProblem
   timesteps            = 40
   duration             = 31.2
   variable names       = (:Ũ⃗, :a, :da, :dda, :Δt)
   knot point dimension = 18

 DirectTrajOptProblem
   timesteps            = 40
   duration             = 31.2
   variable names       = (:Ũ⃗, :a, :da, :dda, :Δt)
   knot point dimension = 18
     DirectTrajOptProblem
   timesteps            = 40
   duration             = 31.2
   variable names       = (:Ũ⃗, :a, :da, :dda, :Δt)
   knot point dimension = 18

 DirectTrajOptProblem
   timesteps            = 40
   duration             = 31.2
   variable names       = (:Ũ⃗, :a, :da, :dda, :Δt)
   knot point dimension = 18
     DirectTrajOptProblem
   timesteps            = 40
   duration             = 31.2
   variable names       = (:Ũ⃗, :a, :da, :dda, :Δt)
   knot point dime

In [7]:
# Adjoint, rollout initialization

∂ₑHₐ = [PAULIS.Z]
varsys_add = VariationalQuantumSystem(
    H_drive,
    ∂ₑHₐ
)
n_seeds = length(seeds)
nQ = length(Qs)

21

In [8]:
# var_prob[1].trajectory.Δt[1], var_prob[1].trajectory.Δt[end ÷ 2]

In [9]:
∂ₑH = [PAULIS.Z]
H_drives = [PAULIS.X, PAULIS.Y, PAULIS.Z]
error_ops = [PAULIS.Z]

function var_obj(
    traj::NamedTrajectory, 
    H_drives::Vector{Matrix{ComplexF64}}, 
    H_errors::Vector{Matrix{ComplexF64}}
)
    Δt = traj.Δt[1]
    varsys = VariationalQuantumSystem(H_drives, H_errors)
    Ũ⃗, ∂Ũ⃗ = variational_unitary_rollout(traj, varsys)

    U = iso_vec_to_operator(Ũ⃗[:, end])
    # First error term
    ∂U = iso_vec_to_operator(∂Ũ⃗[1][:, end])

    d = size(U, 1)
    return abs(tr((U'*∂U)'*(U'*∂U))) / (T * Δt)^2 / d
end

# J_var = var_obj(var_prob.trajectory, H_drives, error_ops)

var_obj (generic function with 1 method)

In [10]:
function tog_obj(
    traj::NamedTrajectory, 
    H_drives::Vector{Matrix{ComplexF64}},
    H_error::Matrix{ComplexF64}
)
    T = traj.T
    Δt = get_timesteps(traj)

    sys = QuantumSystem(H_drives)
    U = iso_vec_to_operator.(eachcol(unitary_rollout(traj, sys)))
    
    # Toggle integral
    H_ti = sum(Δt[i] .* U[i]' * H_error * U[i] for i=1:T-1)

    d₁ = size(U[1], 1)
    Δt₁ = Δt[1]
    metric = norm(tr(H_ti'H_ti)) / (T * Δt₁)^2 / d₁
    return metric
end


tog_obj (generic function with 1 method)

In [11]:
function commutator(A::AbstractMatrix{<:Number}, B::AbstractMatrix{<:Number})
    return A*B - B*A
end

commutator (generic function with 1 method)

In [12]:
function pert_tog_obj(
    traj::NamedTrajectory, 
    H_drives::Vector{Matrix{ComplexF64}},
    H_error::Matrix{ComplexF64};
    order::Int=1,
    a_bound::Float64=a_bound
)
    T = traj.T
    Δt = get_timesteps(traj)

    sys = QuantumSystem(H_drives)
    U = iso_vec_to_operator.(eachcol(unitary_rollout(traj, sys)))

    # toggle integral
    H_ti = zeros(ComplexF64, size(U[1]))

    # note: U_1 = I, so U[:, k] = U_{k-1}.
    # you need to go to T-1, only
    for k in 1:T-1
        Hₖ = sum(traj.a[l, k] / a_bound * H for (l, H) in enumerate(H_drives))
        adjⁿH_E = H_error
        Eₖ_n = H_error * Δt[k]
        
        # get the different orders of the Hadamard lemma
        for n in 2:order
            coef_n = ComplexF64(im^(n-1) * a_bound^(n-1) * Δt[k]^n / factorial(big(n)))
            adjⁿH_E = commutator(Hₖ, adjⁿH_E)
            # Eₖ_n = push!(Eₖ_n, coef_n * adjⁿH_E)
            Eₖ_n += coef_n * adjⁿH_E
        end

        # nth order toggle integral up to k
        H_ti += U[k]' * Eₖ_n * U[k]
    end

    d₁ = size(U[1], 1)
    Δt₁ = Δt[1]
    metric = norm(tr(H_ti'H_ti)) / (T * Δt₁)^2 / d₁
    return metric
end

pert_tog_obj (generic function with 1 method)

In [13]:
# pert_tog_obj(var_prob.trajectory, H_drives, PAULIS.Z, order=1)

In [14]:
var_obj_vals = []
var_objs_sem = []
for (j, Q) in enumerate(Qs)
    var_obj_val_seeds = []
    for (i, seed) in enumerate(seeds)
        Z_vec = vec(var_probs[i,j].trajectory)
        push!(var_obj_val_seeds, var_probs[i,j].objective.L(Z_vec))
    end
    push!(var_obj_vals, mean(var_obj_val_seeds))
    push!(var_objs_sem, std(var_obj_vals) / sqrt(n_seeds))
end

tog_obj_vals = []
tog_objs_sem = []

for (j, Q) in enumerate(Qs)
    tog_obj_val_seeds = []
    for (i, seed) in enumerate(seeds)
        Z_vec = vec(tog_probs[i,j].trajectory)
        push!(tog_obj_val_seeds, tog_probs[i,j].objective.L(Z_vec))
    end
    push!(tog_obj_vals, mean(tog_obj_val_seeds))
    push!(tog_objs_sem, std(tog_obj_vals) / sqrt(n_seeds))
end




In [15]:
"""
    upsample_constant(vals, dts; factor=2)

Take control values `vals` with time steps `dts` (same length),
and upsample by `factor`, returning (vals_up, dts_up).
"""
function upsample_constant_controls(vals::AbstractArray; factor::Int=2)
    vals_up = repeat(vals, inner=factor)
    return vals_up
end

# Example
vals = [1, 2, 3]
dts  = [0.2, 0.2, 0.2]

vals_up = upsample_constant_controls(vals; factor=3)

println(vals_up)  # [1, 1, 2, 2, 3, 3]
#println(dts_up) 
function upsample_matrix(controls::AbstractArray, dts::AbstractArray; factor::Int=2)
    new_controls = []
    for c in eachrow(controls)
        new_c = upsample_constant_controls(c; factor=factor)
        T = length(c)
        push!(new_controls, new_c)
    end
    dts_up = dts[1] / factor .* ones(length(dts) * factor*T)
    new_controls = reduce(vcat, [v' for v in new_controls])
    return new_controls, dts_up
end
function tog_obj_upsample(
    traj::NamedTrajectory, 
    H_drives::Vector{Matrix{ComplexF64}},
    H_error::Matrix{ComplexF64};
    factor::Int=1
)
    T = traj.T * factor
    controls = traj.a
    a_new, Δt_new = upsample_matrix(traj.a, traj.Δt; factor=factor)

    sys = QuantumSystem(H_drives)
    U = iso_vec_to_operator.(eachcol(unitary_rollout(a_new, Δt_new, sys)))
    
    # Toggle integral (truncate at (traj.T - 1) * factor)
    H_ti = sum(Δt_new[i] .* U[i]' * H_error * U[i] for i = 1:(traj.T - 1) * factor)

    d₁ = size(U[1], 1)
    Δt₁ = Δt_new[1]
    metric = norm(tr(H_ti'H_ti)) / (T * Δt₁)^2 / d₁
    return metric
end

[1, 1, 1, 2, 2, 2, 3, 3, 3]


tog_obj_upsample (generic function with 1 method)

In [16]:
# j_vals = 10:10
# length(j_vals)

In [17]:
j = 5
H_errors = [GATES.Z]
t_vec = []#Matrix{Any}(undef, n_js, length(Qs))
v_vec = []#Matrix{Any}(undef, n_js, length(Qs))
t_sem = []
v_sem = []


for (k, Q) in enumerate(Qs)
    t_vec_seed = []
    v_vec_seed = []
    for (i, seed) in enumerate(seeds)
        push!(t_vec_seed, tog_obj_upsample(tog_probs[i,k].trajectory, H_drives, PAULIS.Z; factor=2^j))
        push!(v_vec_seed, tog_obj_upsample(var_probs[i,k].trajectory, H_drives, PAULIS.Z; factor=2^j))
    end
    push!(t_vec, mean(t_vec_seed))
    push!(v_vec, mean(v_vec_seed))

    push!(t_sem, std(t_vec[end]) / sqrt(n_seeds))
    push!(v_sem, std(v_vec[end]) / sqrt(n_seeds))
end


In [18]:
# Process var_probs data (left plot)
vars_objs = []
var_objs_sem = []

for (j, Q) in enumerate(Qs)
    vars_at_Q = [var_obj(var_probs[i,j].trajectory, H_drives, error_ops) for i in 1:n_seeds]
    
    push!(vars_objs, mean(vars_at_Q))
    push!(var_objs_sem, std(vars_at_Q) / sqrt(n_seeds))
end

# Process tog_probs data (right plot)
tog_objs = []
tog_objs_sem = []

for (j, Q) in enumerate(Qs)
    objs_at_Q = [tog_obj(tog_probs[i,j].trajectory, H_drives, error_ops[1]) for i in 1:n_seeds]
    
    push!(tog_objs, mean(objs_at_Q))
    push!(tog_objs_sem, std(objs_at_Q) / sqrt(n_seeds))
end

In [19]:
# script_tog = latexstring("\\mathcal{E}_{T}")
# script_var = latexstring("\\mathcal{E}_{V}")
# script_e = latexstring("\\mathcal{E}")

# normalized_var_objs = vars_objs# ./ Qs
# normalized_tog_objs = tog_objs# ./ Qs
# normalized_v_vec    = v_vec# ./ Qs
# normalized_t_vec    = t_vec# ./ Qs
# # set_theme!(fonts = (; regular = "Times New Roman"))

# # Create a colormap for j values
# colors = Makie.wong_colors()
# # alphas = range(0.3, 1.0, length=n_js)  # Alternative: use alpha for shading

# function scatter_with_line_and_error!(ax, x, y, yerr; color)
#     # Clip lower band at a small positive number for log scale (from target script)
#     lower_band = max.(y .- yerr, 1e-9)
#     upper_band = y .+ yerr
    
#     # Use plotting style from the first (reference) script
#     band!(ax, x, lower_band, upper_band; color=(color, 0.2)) # alpha=0.2
#     lines!(ax, x, y; color=color, linewidth=3) # lw=3
#     scatter!(ax, x, y; color=color, marker=:circle, markersize=10) # markers
#     errorbars!(ax, x, y, yerr; color=color, linewidth=2, whiskerwidth=0) # error bars
# end

# # Helper function to create the custom legend icon from your first script
# function create_custom_legend_icon(color; markersize=10, linewidth=3, errorwidth=2)
#     return [
#         LineElement(color=color, linewidth=linewidth,
#                     points=[Point2f(0.15, 0.5), Point2f(0.85, 0.5)]),  # horizontal line
#         LineElement(color=color, linewidth=errorwidth,
#                     points=[Point2f(0.5, 0.25), Point2f(0.5, 0.75)]),  # vertical error bar
#         MarkerElement(color=color, marker=:circle, markersize=markersize)  # centered dot
#     ]
# end
# # ==============================================================================


# # Create side-by-side plots
# fig = Figure(size=(550, 300))

# # Left plot: Variational-Optimized
# ax1 = Axis(fig[1, 1], 
#     # title="Variational-Optimized Objective Separation", 
#     xlabel=latexstring("Q on ", script_var), 
#     ylabel="Objective Value", 
#     xscale=log10, 
#     yscale=log10)

# scatter_with_line_and_error!(ax1, Qs, normalized_var_objs, var_objs_sem;
#                              color=colors[2])
# scatter_with_line_and_error!(ax1, Qs, normalized_v_vec, v_sem;
#                              color=:gray)

# # Right plot: Toggle-Optimized
# ax2 = Axis(fig[1, 2], 
#     # title="Toggle-Optimized Objective Separation", 
#     xlabel=latexstring("Q on ", script_tog), 
#     xscale=log10, 
#     yscale=log10,)

# hideydecorations!(ax2, ticklabels=true, ticks=true, grid=false) 

# scatter_with_line_and_error!(ax2, Qs, normalized_tog_objs, tog_objs_sem;
#                              color=colors[3])

# scatter_with_line_and_error!(ax2, Qs, normalized_t_vec, t_sem;
#                              color=:gray)

# # # Plot t_vec for all j values with color gradient
# # # (Applying matching linewidth from reference plot, label removed)
# # for (j_idx, j) in enumerate(j_vals)
# #     lines!(ax2, Qs, v_vec;
# #            # label=... removed
# #            linewidth=3,  # Changed from 2 to 3 to match reference style
# #            color=(colors[2], 0.8))
# # end

# # legend parameters
# small_labelsize = 14
# small_patchsize = (25, 18)
# small_margin = (8, 8, 8, 8)
# small_markersize = 6
# small_linewidth = 1.5
# small_errorwidth = 1

# legend_elements_1 = [
#     create_custom_legend_icon(colors[2]; 
#                               markersize=small_markersize, 
#                               linewidth=small_linewidth, 
#                               errorwidth=small_errorwidth),
#     create_custom_legend_icon(:gray; 
#                               markersize=small_markersize, 
#                               linewidth=small_linewidth, 
#                               errorwidth=small_errorwidth)
# ]
# labels_1 = [script_var, latexstring(script_e, " (upsampled)")]

# Legend(fig[1, 1],
#     legend_elements_1,
#     labels_1,
#     halign = :left,
#     valign = :bottom,
#     tellheight = false,
#     tellwidth = false,
#     margin = small_margin,
#     framevisible = false,
#     labelsize = small_labelsize,
#     patchsize = small_patchsize
# )

# legend_elements_2 = [
#     create_custom_legend_icon(colors[3]; 
#                               markersize=small_markersize, 
#                               linewidth=small_linewidth, 
#                               errorwidth=small_errorwidth),
#     create_custom_legend_icon(:gray; 
#                               markersize=small_markersize, 
#                               linewidth=small_linewidth, 
#                               errorwidth=small_errorwidth),
# ]

# labels_2 = [script_tog, latexstring(script_e, " (upsampled)")] 

# Legend(fig[1, 2],
#     legend_elements_2,
#     labels_2,
#     halign = :left,
#     valign = :bottom,
#     tellheight = false,
#     tellwidth = false,
#     margin = small_margin,      # Use smaller margin
#     framevisible = false,
#     labelsize = small_labelsize, # Use smaller label size
#     patchsize = small_patchsize
# )

# fig

# # Optionally save the combined figure
# # save("artifacts/combined_comparison.jpg", fig)

In [24]:
script_tog = latexstring("\\mathcal{E}_{T}")
script_var = latexstring("\\mathcal{E}_{V}")
script_e = latexstring("\\mathcal{E}")

normalized_var_objs = vars_objs# ./ Qs
normalized_tog_objs = tog_objs# ./ Qs
normalized_v_vec    = v_vec# ./ Qs
normalized_t_vec    = t_vec# ./ Qs
# set_theme!(fonts = (; regular = "Times New Roman"))

# Create a colormap for j values
colors = Makie.wong_colors()
# alphas = range(0.3, 1.0, length=n_js)  # Alternative: use alpha for shading

# MODIFIED function signature to accept show_points
function scatter_with_line_and_error!(ax, x, y, yerr; color, linestyle=:solid, show_points::Bool=true)
    # Clip lower band at a small positive number for log scale (from target script)
    lower_band = max.(y .- yerr, 1e-9)
    upper_band = y .+ yerr
    
    # Use plotting style from the first (reference) script
    # Note: If color is a tuple like (:gray, 0.7), the band alpha will be 0.7 * 0.2 = 0.14
    band!(ax, x, lower_band, upper_band; color=(color, 0.2)) 
    
    # The passed `color` (e.g., (:gray, 0.7)) will be used here
    lines!(ax, x, y; color=color, linewidth=3, linestyle=linestyle) 
    
    # MODIFIED: Condition now uses show_points
    if show_points
        scatter!(ax, x, y; color=color, marker=:circle, markersize=10) # markers
    end
    
    # The passed `color` will also be used for error bars
    errorbars!(ax, x, y, yerr; color=color, linewidth=2, whiskerwidth=0) # error bars
end

# Helper function to create the custom legend icon from your first script
# MODIFIED function signature to accept show_points
function create_custom_legend_icon(color; markersize=10, linewidth=3, errorwidth=2, linestyle=:solid, show_points::Bool=true)
    # Initialize as Vector{Any} to hold mixed element types
    elements = Any[
        LineElement(color=color, linewidth=linewidth,
                    points=[Point2f(0.15, 0.5), Point2f(0.85, 0.5)], linestyle=linestyle), # horizontal line
        LineElement(color=color, linewidth=errorwidth,
                    points=[Point2f(0.5, 0.25), Point2f(0.5, 0.75)])  # vertical error bar
    ]
    
    # MODIFIED: Condition now uses show_points
    if show_points
        push!(elements, MarkerElement(color=color, marker=:circle, markersize=markersize)) # centered dot
    end
    
    return elements
end
# ==============================================================================


# Create side-by-side plots
fig = Figure(size=(450, 300))

# Left plot: Toggle-Optimized
ax1 = Axis(fig[1, 1], 
    # title="Toggle-Optimized Objective Separation", 
    xlabel=latexstring("Q on ", script_tog), 
    ylabel="Objective Value",
    xscale=log10, 
    yscale=log10,)

# This plot will use the defaults (linestyle=:solid, show_points=true)
scatter_with_line_and_error!(ax1, Qs, normalized_tog_objs, tog_objs_sem;
                            color=colors[3])

# MODIFIED call: Set transparent color, solid line, and no points
scatter_with_line_and_error!(ax1, Qs, normalized_t_vec, t_sem;
                            color=(:gray, 0.7), 
                            linestyle=:solid, 
                            show_points=false)

# Right plot: Variational-Optimized
ax2 = Axis(fig[1, 2],
    # title="Variational-Optimized Objective Separation", 
    xlabel=latexstring("Q on ", script_var), 
    xscale=log10, 
    yscale=log10)

hideydecorations!(ax2, ticklabels=true, ticks=true, grid=false) 

# This plot will use the defaults (linestyle=:solid, show_points=true)
scatter_with_line_and_error!(ax2, Qs, normalized_var_objs, var_objs_sem;
                            color=colors[2])
                            
# MODIFIED call: Set transparent color, solid line, and no points
scatter_with_line_and_error!(ax2, Qs, normalized_v_vec, v_sem;
                            color=(:gray, 0.7), 
                            linestyle=:solid, 
                            show_points=false)


# legend parameters
small_labelsize = 14
small_patchsize = (25, 18)
small_margin = (8, 8, 8, 8)
small_markersize = 6
small_linewidth = 1.5
small_errorwidth = 1

legend_elements_1 = [
    create_custom_legend_icon(colors[3]; 
                                markersize=small_markersize, 
                                linewidth=small_linewidth, 
                                errorwidth=small_errorwidth), # Uses defaults (solid, show_points)
    # MODIFIED call: Set transparent color, solid line, and no points
    create_custom_legend_icon((:gray, 0.7); 
                                markersize=small_markersize, 
                                linewidth=small_linewidth, 
                                errorwidth=small_errorwidth,
                                linestyle=:solid,
                                show_points=false),
]

labels_1 = [script_tog, latexstring(script_e, " (upsampled)")] 

Legend(fig[1, 1],
    legend_elements_1,
    labels_1,
    halign = :left,
    valign = :bottom,
    tellheight = false,
    tellwidth = false,
    margin = small_margin,      # Use smaller margin
    framevisible = false,
    labelsize = small_labelsize, # Use smaller label size
    patchsize = small_patchsize
)
legend_elements_2 = [
    create_custom_legend_icon(colors[2]; 
                                markersize=small_markersize, 
                                linewidth=small_linewidth, 
                                errorwidth=small_errorwidth), # Uses defaults (solid, show_points)
    # MODIFIED call: Set transparent color, solid line, and no points
    create_custom_legend_icon((:gray, 0.7); 
                                markersize=small_markersize, 
                                linewidth=small_linewidth, 
                                errorwidth=small_errorwidth,
                                linestyle=:solid,
                                show_points=false)
]

labels_2 = [script_var, latexstring(script_e, " (upsampled)")]

Legend(fig[1, 2],
    legend_elements_2,
    labels_2,
    halign = :left,
    valign = :bottom,
    tellheight = false,
    tellwidth = false,
    margin = small_margin,
    framevisible = false,
    labelsize = small_labelsize,
    patchsize = small_patchsize
)

fig

# Optionally save the combined figure
save("artifacts/combined_comparison.png", fig)

In [21]:
# display(CairoMakie.plot(var_probs[end, end].trajectory, [:Ũ⃗, :a, :da, :dda]))
# display(CairoMakie.plot(htog_probs[end, end].trajectory, [:Ũ⃗, :a, :da, :dda]))
